In [1]:
pip install pandas fastparquet

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   --------------------------------- ------ 1.8/2.2 MB 10.0 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 9.3 MB/s eta 0:00:00
  Created wheel for fastparquet: filename=fastparquet-2024.11.0-cp312-cp312-win_amd64.whl size=672772 sha256=86791781aba55ec4d96658873b396fcf311dc0e7f7dfd0b4c0f7a095efea6ae4
  Stored in directory: c:\users\msfal\appdata\local\pip\cache\wheels\10\6d\17\0e26bf80f5639ab5e79745c7985af4e869cc8b99adc606fa40
Successfully built fastparquet
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

# URL for the lookup table
URL_DATA = 'https://storage.data.gov.my/pricecatcher/lookup_item.parquet'

# Read the parquet file into a DataFrame
df_lookup = pd.read_parquet(URL_DATA)

# Convert 'date' column to datetime if it exists
if 'date' in df_lookup.columns:
    df_lookup['date'] = pd.to_datetime(df_lookup['date'])

# Display the DataFrame
print(df_lookup.head())

   item_code                                               item  unit  \
0         -1                                               None  None   
1          1                             AYAM BERSIH - STANDARD   1kg   
2          2                                AYAM BERSIH - SUPER   1kg   
3          3                                         AYAM HIDUP   1kg   
4          9  DAGING KAMBING BEBIRI IMPORT BERTULANG (MUTTON...   1kg   

       item_group item_category  
0            None          None  
1  BARANGAN SEGAR          AYAM  
2  BARANGAN SEGAR          AYAM  
3  BARANGAN SEGAR          AYAM  
4  BARANGAN SEGAR        DAGING  


In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import requests
from io import BytesIO
from datetime import datetime, timedelta

# Function to generate URLs for each month
def generate_price_data_urls(start_date, end_date):
    urls = []
    current = start_date.replace(day=1)
    while current <= end_date:
        url = f"https://storage.data.gov.my/pricecatcher/pricecatcher_{current.strftime('%Y-%m')}.parquet"
        urls.append(url)
        # Move to the next month
        if current.month == 12:
            current = current.replace(year=current.year + 1, month=1)
        else:
            current = current.replace(month=current.month + 1)
    return urls

# Define the date range for price data
start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 11, 1)  # Assuming data is up to November 2024

# Generate all URLs
price_data_urls = generate_price_data_urls(start_date, end_date)

# Initialize an empty list to store DataFrames
price_data_list = []

# Fetch and load each parquet file
for url in price_data_urls:
    try:
        print(f"Fetching data from: {url}")
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status
        df = pd.read_parquet(BytesIO(response.content))
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
        price_data_list.append(df)
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred for {url}: {http_err}")
    except Exception as err:
        print(f"Other error occurred for {url}: {err}")

# Concatenate all monthly DataFrames into a single DataFrame
df_price = pd.concat(price_data_list, ignore_index=True)

# Display the combined DataFrame
print(df_price.head())
print(f"Total records fetched: {len(df_price)}")

Fetching data from: https://storage.data.gov.my/pricecatcher/pricecatcher_2022-01.parquet
Fetching data from: https://storage.data.gov.my/pricecatcher/pricecatcher_2022-02.parquet
Fetching data from: https://storage.data.gov.my/pricecatcher/pricecatcher_2022-03.parquet
Fetching data from: https://storage.data.gov.my/pricecatcher/pricecatcher_2022-04.parquet
Fetching data from: https://storage.data.gov.my/pricecatcher/pricecatcher_2022-05.parquet
Fetching data from: https://storage.data.gov.my/pricecatcher/pricecatcher_2022-06.parquet
Fetching data from: https://storage.data.gov.my/pricecatcher/pricecatcher_2022-07.parquet
Fetching data from: https://storage.data.gov.my/pricecatcher/pricecatcher_2022-08.parquet
Fetching data from: https://storage.data.gov.my/pricecatcher/pricecatcher_2022-09.parquet
Fetching data from: https://storage.data.gov.my/pricecatcher/pricecatcher_2022-10.parquet
Fetching data from: https://storage.data.gov.my/pricecatcher/pricecatcher_2022-11.parquet
Fetching d

In [5]:
df_price.to_csv('pricecatcher_2022_2024.csv', index=False)

KeyboardInterrupt: 